In [23]:
#brainfuck parser v2

braincleanse = """
a=15;
b=6;
c = a/b;
c += 48;
@rem += 48;
print(c);
delete(c);
print(@rem);
"""

dataPointer = 0
#all other are guaranteed to be 0
#@ is for temp variables
#@rem is the remainder after division operation
#print prints the variable inside
#delete deletes the variable inside
variables = {}
brainfuckCommands = []

#cannot use @ and |
validOperators = ["=", "+", "-", "*", "/", "%", "^", ",", ";", ":", "[", "]", "{", "}", "(", ")"]

def codeError(msg):
    print("ERROR: " + msg)

def isnumber(x):
    try:
        int(x)
        return True
    except ValueError:
        return False
    
#code checks before compilation to brainfuck
def staticCheck(i):
    original = i + ";"
    string = i
    for i in validOperators:
        string = string.replace(i, "|")
    l = string.split("|")
    r = []
    for i in l:
        if not isnumber(i) and i is not "":
            r.append(i)  
    if len(r) != len(set(r)):
        #contains duplicates
        codeError(original + "\ncontains duplicate variables. Each variable can be used only one in one line")
        1/0

braincleanse = braincleanse.replace('\n',"")
dirtyLines = braincleanse.split(";")
lines = []
for i in dirtyLines:
    if (i != "" and ("=" in i or "(" in i or "{" in i or "}" in i) and not "#" in i):
        i = i.replace(" ", "")
        staticCheck(i)
        lines.append(i)
        
def allocateVariableAt(var, preferredIndex, size):
    #chance to fill previous
    cell = preferredIndex - size
    while True:
        if cell < 0:
            cell = 0
            continue
        free = True
        for x in range(size):
            for i in variables:
                if variables[i]["index"] == cell + x:
                    free = False
                    break
        if free:
            realloc = var in variables
            variables[var] = {"index" : cell, "size": size}
            print("reallocating" if realloc else "allocating new" + " variable:", var, "at location:", cell)
            break
        cell += 1
#around dataPointer
def allocateVariable(var, size):
    allocateVariableAt(var, dataPointer, size)
        
#for temps
def freeVariable(var):
    del variables[var]
    print("freed variable:", var)

def parseLine(string):
    found = string.find("=")
    specialOperators = ['+', '-']
    if found != -1:
        if string[found - 1] in specialOperators:
            return (string[0:found - 1], string[found - 1:])
        else:
            return (string[0:found], string[found+1:])
    else:
        return("", string)
    
def fetchData(index):
    global dataPointer
    assert(index >= 0)
    move = index - dataPointer
    dataPointer = index
    if move < 0:
        brainfuckCommands.append("<" * (move * -1))
    else:
        brainfuckCommands.append(">" * move)
        
def changeData(n):
    if n < 0:
        brainfuckCommands.append("-" * (n * -1))
    else:
        brainfuckCommands.append("+" * n)
    
###BEGIN OPERATORS###
def increment(context):
    if context[1].startswith("+="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '+' increment
        if isnumber(context[1]):
            assignConstant(context)
        else:
            allocateVariable("@")
            fetchData(variables[context[1]]["index"])
            brainfuckCommands.append("[")
            fetchData(variables[context[0]]["index"])
            brainfuckCommands.append("+")
            fetchData(variables["@"]["index"])
            brainfuckCommands.append("+")
            fetchData(variables[context[1]]["index"])
            brainfuckCommands.append("-]")
            
            variables[context[1]] = variables["@"]
            freeVariable("@")
        return True
    return False
            
def decrement(context):
    if context[1].startswith("-="):
        context = (context[0],context[1][2:], context[2])
        #if not numeric, do this, else do '-' decrement
        if isnumber(context[1]):
            if not context[1].startswith('-'):
                context = (context[0],"-" + context[1])
            else:
                context = (context[0],context[1][1:])
            assignConstant(context)
        else:
            xIndex = variables[context[1]]
            allocateVariable(context[1])
            
            fetchData(xIndex)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]]["index"])
            brainfuckCommands.append("-")
            fetchData(variables[context[1]]["index"])
            brainfuckCommands.append("+")
            fetchData(xIndex)
            brainfuckCommands.append("-]")
        return True
    return False

def add(context):
    plusIndex = context[1].find("+")
    if plusIndex != -1:
        x0 = context[1][:plusIndex]
        x1 = context[1][plusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]]["index"])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        increment((context[0], "+=" + x1, context[2]))
        return True
    return False
        
def substract(context):
    minusIndex = context[1].find("-")
    if minusIndex != -1:
        x0 = context[1][:minusIndex]
        x1 = context[1][minusIndex + 1:]
        if x0 == "" or x1 == "":
            return False
        #clear if previously held value
        if context[2]:
            fetchData(variables[context[0]]["index"])
            brainfuckCommands.append("[-]")
        increment((context[0], "+=" + x0, context[2]))
        decrement((context[0], "-=" + x1, context[2]))
        return True
    return False

def getInput(context):
    if context[1].find("input(") != -1:
        expression = context[1][6:-1]
        allocateVariable(expression)
        fetchData(variables[expression]["index"])
        brainfuckCommands.append(',')
        return True
    return False

def printOutput(context):
    if context[1].find("print(") != -1:
        expression = context[1][6:-1]
        if isnumber(expression):
            allocateVariable("@")
            fetchData(variables["@"]["index"])
            changeData(int(expression))
            freeVariable("@")
        else:
            fetchData(variables[expression]["index"])
        brainfuckCommands.append('.')
        return True
    return False

def multiply(context):
    multiplyIndex = context[1].find("*")
    if multiplyIndex != -1:
        x0 = context[1][:multiplyIndex]
        x1 = context[1][multiplyIndex + 1 :]
        x0type = isnumber(x0) #true if is number
        x1type = isnumber(x1)
        
        def numMultNum(num0, num1):
            if num1 < num0:
                num0, num1 = num1, num0
            #num0 is always smaller here
            allocateVariable("@")
            fetchData(variables["@"]["index"])
            changeData(num0)
            brainfuckCommands.append("[")
            fetchData(variables[context[0]]["index"])
            changeData(num1)
            fetchData(variables["@"]["index"])
            brainfuckCommands.append("-]")
            freeVariable("@")
            
        def numMultVar(num, var):
            previousVarPosition = variables[var]["index"]
            allocateVariable(var)
            fetchData(previousVarPosition)
            brainfuckCommands.append("[")
            fetchData(variables[var]["index"])
            brainfuckCommands.append("+")
            fetchData(variables[context[0]]["index"])
            changeData(num)
            fetchData(previousVarPosition)
            brainfuckCommands.append("-]")
            
        def varMultVar(var0, var1):
            #really important its as close as possible
            allocateVariableAt("@", variables[var1]["index"])
            allocateVariableAt("@@", variables[var0]["index"])
            
            #do this var0 times
            fetchData(variables[var0]["index"])
            brainfuckCommands.append("[")
            fetchData(variables["@@"])
            brainfuckCommands.append("+")
            #outer loop start
            #inner loop, add var1 to result and move var1
            #add var1 to result
            fetchData(variables[var1]["index"])
            brainfuckCommands.append("[")
            fetchData(variables["@"]["index"])
            brainfuckCommands.append("+")
            fetchData(variables[context[0]])
            brainfuckCommands.append("+")
            fetchData(variables[var1]["index"])
            brainfuckCommands.append("-]")
            #move value from @ to var1
            fetchData(variables["@"]["index"])
            brainfuckCommands.append("[")
            fetchData(variables[var1])
            brainfuckCommands.append("+")
            fetchData(variables["@"]["index"])
            brainfuckCommands.append("-]")
            #outer loop end
            fetchData(variables[var0]["index"])
            brainfuckCommands.append("-]")
            
            #var0 moved to @@
            variables[var0] = variables["@@"]
            
            #cleanup
            freeVariable("@")
            freeVariable("@@")
        
        if x0type and x1type:
            numMultNum(int(x0), int(x1))
        elif not x0type and x1type:
            numMultVar(int(x1), x0)
        elif x0type and not x1type:
            numMultVar(int(x0), x1)
        else:
            varMultVar(x0, x1)
        
        return True
    return False

def divide(context):
    divideIndex = context[1].find("/")
    if divideIndex != -1:
        x0 = context[1][:divideIndex]
        x1 = context[1][divideIndex + 1 :]
        x0type = isnumber(x0) #true if is number
        x1type = isnumber(x1)
        
        def numDivNum(num0, num1):
            pass
        def numDivVar(num, var):
            pass
        def varDivVar(var0, var1):
            pass
        
        if x0type and x1type:
            numDivNum(int(x0), int(x1))
        elif not x0type and x1type:
            numDivVar(int(x1), x0)
        elif x0type and not x1type:
            numDivVar(int(x0), x1)
        else:
            varDivVar(x0, x1)
        
        return True
    return False

def delete(context):
    if context[1].find("delete(") != -1:
        expression = context[1][7:-1]
        if expression in variables:
            freeVariable(expression)
        return True
    return False

#always last!
def assignConstant(context):
    if isnumber(context[1]):
        fetchData(variables[context[0]]["index"])
        changeData(int(context[1]))
    else:
        increment((context[0], "+=" + context[1], context[2]))
    return True
###END OPERATORS###
    
for i in lines:
    parsed = parseLine(i)
    if parsed[0] != "" and parsed[0] not in variables:
        allocateVariable(parsed[0], 1)
        parsed = (parsed[0], parsed[1], False)
    else:
        parsed = (parsed[0], parsed[1], True)
        #certainly clear result if needed
        if parsed[0] != "":
            fetchData(variables[parsed[0]]["index"])
            brainfuckCommands.append("[-]")
    
    #check for possible operations
    if increment(parsed):
        continue
    if decrement(parsed):
        continue
    if add(parsed):
        continue
    if getInput(parsed):
        continue
    if printOutput(parsed):
        continue
    if multiply(parsed):
        continue
    if divide(parsed):
        continue
    if delete(parsed):
        continue
    if substract(parsed):
        continue
    if assignConstant(parsed):
        continue
    
brainfuck = "".join(brainfuckCommands)
print("\n\n######################OUTPUT#########################")
print(brainfuck)

'''# >n 0 d
[->+>-[>+>>]>[+[-<+>]>+>>]<<<<<<]
# >0 n d-n%d n%d n/d'''

allocating new variable: a at location: 0
allocating new variable: b at location: 1
allocating new variable: c at location: 2
allocating new variable: @rem at location: 3
freed variable: c


######################OUTPUT#########################
+++++++++++++++>++++++>[-]++++++++++++++++++++++++++++++++++++++++++++++++>++++++++++++++++++++++++++++++++++++++++++++++++<.>.


'# >n 0 d\n[->+>-[>+>>]>[+[-<+>]>+>>]<<<<<<]\n# >0 n d-n%d n%d n/d'